In [4]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [5]:
def wait1(s):
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def call(f, *args):
    s = "(" + ", ".join(map(repr, args)) + ")"
    print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        else:
            return rv
        
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

def setdo(mode, n2_or_sp, o2=None):
    call(h.setdo, mode, n2_or_sp,o2)
    
def doman(n2, o2):
    setdo(1, n2, o2)
    
def doauto(sp):
    setdo(0, sp)
    
def dooff():
    setdo(2, 0, 0)
        
def endbatch():
    call(h.endbatch)

def wait2(s):
    pass
wait = wait1

In [7]:
h = HelloApp('71.189.82.196:85')
wait = wait1
dooff()
wait(3600*10)
startbatch("dopid 1.1")
doauto(150)
wait(3600*20)
endbatch()
startbatch("dopid 1.2")
doauto(100)
wait(3600*20)
endbatch()
startbatch("dopid 1.3")
doauto(50)
wait(3600*20)
endbatch()

Calling setdo(2, 0, 0)
Sleeping 36000 seconds, ending at 05/06/2017 04:02:38 AM
Calling batchrunning()
Calling startbatch('dopid 1.1')
Calling setdo(0, 150, None)
Sleeping 72000 seconds, ending at 05/07/2017 12:02:44 AM
Calling endbatch()
Calling batchrunning()
Calling startbatch('dopid 1.2')
Calling setdo(0, 100, None)
Sleeping 72000 seconds, ending at 05/07/2017 08:02:50 PM
Calling endbatch()
Calling batchrunning()
Calling startbatch('dopid 1.3')
Calling setdo(0, 50, None)
Sleeping 72000 seconds, ending at 05/08/2017 04:02:53 PM
Calling endbatch()


## Download the Spoils of War

In [11]:
h = HelloApp('71.189.82.196:85')
p='C:\\PBSCloudStation\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy\\PID Tuning\\DO\\Raw Data\\'
def dl(bn):
    b = call(h.getdatareport_bybatchname, bn)
    with open(p+bn+".csv", 'wb') as f:
        f.write(b)

In [12]:
for i in (1,2,3):
    dl("dopid 1.%d"%i)

Calling getdatareport_bybatchname('dopid 1.1')
Calling getdatareport_bybatchname('dopid 1.2')
Calling getdatareport_bybatchname('dopid 1.3')


## Analysis

In [13]:
files = [p + bn + '.csv' for bn in ['dopid 1.%d'%d for d in (1,2,3)]]

In [60]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def open_file(file):
    global wb, ws, cells, cr
    wb = xl.Workbooks.Open(file)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    x, y = chart_rng(c)
    CreateDataSeries(chart, x, y, v)
    
    
def analyze():
    global c1
    c1 = cells.Find("DOPV(%)")
    ref = c1 = c1.Offset(2,1)
    mk_elapsed(c1, ref)
    ref.Offset(0, 2).Value = "1440"
    mkch(c1,"DOPV(%)")
    
    for v in "DON2FlowActualRequest(%)", "DOO2FlowControllerRequestLimited(%)":
        c = cells.Find(v).Offset(2,1)
        mk_elapsed(c, ref)
        mkch(c,v)
        
    

In [61]:
xl = Excel()
open_file(files[0])
analyze()

In [47]:
c1.Address

'$BQ$2'